In [110]:
##加载模型
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
tokenizer = AutoTokenizer.from_pretrained("detected_model")
model = AutoModelForCausalLM.from_pretrained("detected_model", torch_dtype=torch.bfloat16)

In [111]:
#加载数据集
import os
os.environ['TOKENIZERS_PARALLELISM'] = "True"
flatten = lambda l : [x for s in l for x in s]
shuffle = lambda l : random.sample(l, k=len(l))
data_path='dataset/valid.json'
print("loading from json...")
with open(data_path, 'r') as f:
    data = f.read()
    lines=json.loads(data)
tokens=[d['text'] for d in lines]

loading from json...


In [112]:

#对tokens[0]进行实验，查看一下效果
token=tokens[0].split()
print(token)
#本来想取一半的，考虑到模型本身的能力，改成3/4（这个地方可以多试试，我也不确定取什么好）
quarter_index=len(token)//4

input=' '.join(token[:quarter_index])
input_text=input.replace('Ġ','')
input_id=tokenizer.encode(input_text, return_tensors='pt')
output=model.generate(input_id, max_length=len(token),num_return_sequences=1, temperature=0.7)
output_text=tokenizer.decode(output[0], skip_special_tokens=True)
output_text=output_text.replace('Ġ','')
print("input:",input_text)
print("output:",output_text)

['A', 'water', 'company', 'has', 'blamed', 'more', 'people', 'working', 'from', 'home', 'post-pandemic', 'for', 'a', 'new', 'hosepipe', 'ban.South', 'East', 'Water,', 'which', 'supplies', 'more', 'than', '2m', 'homes', 'and', 'businesses,', 'will', 'impose', 'the', 'first', 'hosepipe', 'ban', 'of', 'the', 'summer', 'on', 'Monday,', 'affecting', 'households', 'across', 'Kent', 'and', 'Sussex.The', 'company’s', 'chief', 'executive,', 'David', 'Hinton,', 'said', 'that', 'people', 'working', 'from', 'home', 'was', 'a', '“key', 'factor”', 'behind', 'the', 'ban,', 'as', 'it', 'has', '“increased', 'drinking', 'water', 'demand”.In', 'a', 'letter', 'to', 'customers,', 'he', 'wrote:', '“Over', 'the', 'past', 'three', 'years', 'the', 'way', 'in', 'which', 'drinking', 'water', 'is', 'being', 'used', 'across', 'the', 'south-east', 'has', 'changed', 'considerably.“The', 'rise', 'of', 'working', 'from', 'home', 'has', 'increased', 'drinking', 'water', 'demand', 'in', 'commuter', 'towns', 'by', 'aroun

/Users/liuxingyu/anaconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


input: A water company has blamed more people working from home post-pandemic for a new hosepipe ban.South East Water, which supplies more than 2m homes and businesses, will impose the first hosepipe ban of the summer on Monday, affecting households across Kent and Sussex.The company’s chief executive, David Hinton, said that people working from home was a “key factor” behind the ban, as it has “increased drinking water demand”.In a letter to customers, he wrote: “Over the past three years the way in which drinking water is being used across the south-east has changed considerably.“The rise of working from home has increased drinking water demand in commuter towns by around 20% over a very short period, testing our existing infrastructure.”Hinton also blamed low rainfall since April as well as a recent spell of hot weather which he said led to a spike in demand for drinking water.“Our reservoir and aquifer stocks of raw water, essential to our water supply but
output: A water company h

In [113]:
#评估相似度(使用SequenceMatcher)（效果不是很好，不能很好地区分）
from difflib import SequenceMatcher
matcher=SequenceMatcher(None,tokens[0], output_text)
similarity=matcher.ratio()
print(similarity)

0.3722764838467318


In [116]:
#评估相似度(使用余弦相似度) 莫名其妙很高，我还没发现哪里有问题
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# 将tokens转换为token ids
token_ids1 = tokenizer.encode(' '.join(tokens[2]), return_tensors='pt')
token_ids2 = tokenizer.encode(' '.join(output_text), return_tensors='pt')

# 将token ids转换为向量
vec1 = model.get_input_embeddings()(token_ids1).to(dtype=torch.float32).detach().numpy()
vec2 = model.get_input_embeddings()(token_ids2).to(dtype=torch.float32).detach().numpy()

# 将三维张量转换为二维张量
vec1 = np.mean(vec1, axis=1)
vec2 = np.mean(vec2, axis=1)

# 计算vec1和vec2的余弦相似度
similarity = cosine_similarity(vec1, vec2)

print(similarity)

[[0.9987321]]


In [117]:
#喂给bert看cls的相似度 也感觉有点高
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# 初始化tokenizer和model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


# 使用tokenizer对句子进行编码，然后将编码的结果喂给model
inputs1 = tokenizer(tokens[0], return_tensors='pt', truncation=True, max_length=512, padding='max_length')
inputs2 = tokenizer(output_text, return_tensors='pt', truncation=True, max_length=512, padding='max_length')

with torch.no_grad():
    outputs1 = model(**inputs1)
    outputs2 = model(**inputs2)

# 提取出CLS标记
cls1 = outputs1[0][0][0].numpy()
cls2 = outputs2[0][0][0].numpy()

# 计算cls1和cls2的余弦相似度
similarity = cosine_similarity(cls1.reshape(1, -1), cls2.reshape(1, -1))

print(similarity)

[[0.8943842]]
